### Импорт библиотек для работы с бд и дф

In [2]:
import psycopg2
import pandas as pd
import sqlalchemy

### Кредиты для коннекта к бд

In [13]:
conn = psycopg2.connect(host="127.0.0.1", port="5433", database="online_store", 
                            user="admin", password="root")

### Выдаёт «повёрнутую таблицу», содержащую имена строк плюс N столбцов значений

In [4]:
cur = conn.cursor()
cur.execute('''
            CREATE EXTENSION tablefunc;
            ''')

cur.execute('''
           CREATE TABLE sales(year int, month int, qty int);
            ''')

cur.execute('''
           INSERT INTO sales VALUES
           (2007, 1, 1000),
           (2007, 2, 1500),
           (2007, 7, 500), 
           (2007, 11, 1500),
           (2007, 12, 2000),
           (2008, 1, 1000),
           (2009, 5, 2500),
           (2009, 9, 800);

            ''')

cur.execute('''
           SELECT * FROM sales;
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['year', 'month', 'qty'])

,year,month,qty
0,2007,1,1000
1,2007,2,1500
2,2007,7,500
3,2007,11,1500
4,2007,12,2000
5,2008,1,1000
6,2009,5,2500
7,2009,9,800


In [8]:
cur.execute('''
           SELECT * FROM crosstab(
          -- два запроса особой структуры :
          -- необходимо использовать "ORDER BY 1" 
          $$ SELECT year, month, qty FROM sales ORDER BY 1 $$, 
          
          -- Второй параметр может быть любым запросом, 
          -- который возвращает по одной строке для каждого атрибута, 
          -- соответствующего порядку определения столбца в конце. 
          $$ SELECT m FROM generate_series(1,12) m $$ 
        ) AS ( -- квотирование в долларах, чтобы упростить цитирование. 
          year int, "Jan" int, "Feb" int, "Mar" int, "Apr" int, 
          "May" int, "Jun" int, "Jul" int, "Aug" int, "Sep" int, 
          "Oct" int, "Nov" int, "Dec" int
        );
            ''')

In [9]:

results = cur.fetchall()
pd.DataFrame(results, columns=['year', 'Jan', 'Feb',
                              'Mar', 'Apr', 'May',
                               'Jun', 'Jul', 'Aug',
                               'Sep', 'Oct', 'Nov',
                               'Dec'
                              ])

,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2007,1000.0,1500.0,None,None,NaN,None,500.0,None,NaN,None,1500.0,2000.0
1,2008,1000.0,NaN,None,None,NaN,None,NaN,None,NaN,None,NaN,NaN
2,2009,NaN,NaN,None,None,2500.0,None,NaN,None,800.0,None,NaN,NaN


### FILTER PIVOT

In [23]:
cur = conn.cursor()
cur.execute('''
            CREATE TEMP TABLE t4 (
            EFF_DATE date,
            VERSION_ID character,
            amnt integer
            )
            ''')

cur.execute('''
            INSERT INTO t4 VALUES  
            ('2012-01-01', 1, 10),
            ('2012-01-01', 0, 20),
            ('2012-02-01', 1, 30),
            ('2012-02-01', 0, 40)
            ''')

cur.execute('''
            SELECT 
            VERSION_ID,
            sum(amnt) FILTER (WHERE EFF_DATE = '2012-01-01') as jun,
            sum(amnt) FILTER (WHERE EFF_DATE = '2012-02-01') as feb
            from t4
            group by VERSION_ID
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['VERSION_ID', 'jun', 'feb'])

,VERSION_ID,jun,feb
0,0,20,40
1,1,10,30


In [21]:
conn.rollback()
conn.close()